# Notebook de Testing

## Bibliotecas

In [3]:
#Básicos para manipulacion de datos 
import pandas as pd
import numpy as np
#Graficas 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
#Auxiliares
from tqdm import tqdm
from itertools import compress
from sklearn.preprocessing import MinMaxScaler
#Optimización multiobjetivo 
from pymoo.optimize import minimize
from pymoo.core.repair import Repair
#Algoritmos evolutivos
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.age import AGEMOEA
from pymoo.algorithms.moo.sms import SMSEMOA
#Operadores 
from pymoo.visualization.scatter import Scatter
#Features pymoo api 
from pymoo.problems import get_problem
from pymoo.termination import get_termination
from pymoo.problems import get_problem
from pymoo.util.ref_dirs import get_reference_directions
#Indicadores
from pymoo.indicators.hv import HV
#Población (soluciones)
from pymoo.core.population import Population

#Plugins 
from Plugins import pymoo_utils
from Plugins import archieving_strategies
from Plugins import operators

c:\Users\yeudi\AppData\Local\Programs\Python\Python310\lib\site-packages\pymoo\algorithms\moo\age.py:212: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(fastmath=True)
c:\Users\yeudi\AppData\Local\Programs\Python\Python310\lib\site-packages\pymoo\algorithms\moo\age.py:222: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(fastmath=True)
c:\Users\yeudi\AppDa

## Problems Definitions

In [4]:
#WFG Problems 
wfg_problems = ['wfg1', 'wfg2', 'wfg3', 'wfg4', 'wfg5', 'wfg6', 'wfg7', 'wfg8', 'wfg9']
wfg_vars     = [24]*len(wfg_problems)
wfg_obj      = [3]*len(wfg_problems)
#DTLZ Problems 
dtlz_problems = ['dtlz1', 'dtlz2', 'dtlz3', 'dtlz4', 'dtlz5', 'dtlz6', 'dtlz7']
dtlz_vars = [12]*len(dtlz_problems)
dtlz_obj  = [3]*len(dtlz_problems)
#ZDT Problems 
zdt_problems = ['zdt1', 'zdt2', 'zdt3', 'zdt4', 'zdt6']
zdt_vars     = [np.nan]*len(zdt_problems)
zdt_obj      = [np.nan]*len(zdt_problems)
#General Problems
general_problems = wfg_problems+ dtlz_problems + zdt_problems
general_vars     = wfg_vars + dtlz_vars + zdt_vars
general_obj      = wfg_obj + dtlz_obj + zdt_obj

Tiempos de ejecución predefinidos

In [ ]:
n_ejec = 20
n_gen  = 250

# NSGA-II

## Baseline 

In [ ]:
pymoo_utils.get_solutions_fake(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
              {'pop_size':100, 'repair': operators.Simplex_Repair()}, 
              name_oprt='baseline')

## Samplings 

In [ ]:
pymoo_utils.get_solutions_fake(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
               {'pop_size':100,'sampling': operators.Sampling_Energy(), 'repair': operators.Simplex_Repair()}, 
               name_oprt='sam_energy')
pymoo_utils.get_solutions_fake(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
               {'pop_size':100,'sampling': operators.Sampling_Uniform(), 'repair': operators.Simplex_Repair()}, 
               name_oprt='sam_uniform')
pymoo_utils.get_solutions_fake(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
               {'pop_size':100,'sampling': operators.Sampling_RED_D(), 'repair': operators.Simplex_Repair()}, 
               name_oprt='sam_redd')
pymoo_utils.get_solutions(general_problems, general_vars, general_obj,n_ejec, n_gen, 'nsgaii', 
               {'pop_size':100,'sampling': operators.Sampling_MSS_D(), 'repair': operators.Simplex_Repair()}, 
               name_oprt='sam_mssd')

## Mutations

In [ ]:
pymoo_utils.get_solutions_fake(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
               {'pop_size':100,'mutation': operators.Probability_Mutation(operators.mut_dirichlet), 'repair': operators.Simplex_Repair()}, 
               name_oprt='mut_dirichlet')
pymoo_utils.get_solutions(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
                {'pop_size':100,'mutation': operators.Probability_Mutation(operators.mut_lognormal_transform), 'repair': operators.Simplex_Repair()}, 
                name_oprt='mut_lognormal')
pymoo_utils.get_solutions(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
                {'pop_size':100,'mutation': operators.Probability_Mutation(operators.mut_exponential_transform), 'repair': operators.Simplex_Repair()}, 
                name_oprt='mut_exponential')
pymoo_utils.get_solutions(general_problems, general_vars, general_obj, n_ejec, n_gen, 'nsgaii', 
                {'pop_size':100,'mutation': operators.Probability_Mutation(operators.mut_normal_transform), 'repair': operators.Simplex_Repair()}, 
                name_oprt='mut_normal')

# Review NaN errors

In [1]:
def checkingNaNs(problems, oprts, name_algorithm): 
    count_missing = 0
    count_error_shape = 0 
    for problem in problems: 
        for name_oprt in oprts: 
            Mega_X = np.load('Solutions/'+name_algorithm+'/'+problem+'_'+name_oprt+'_X.npy')
            Mega_F = np.load('Solutions/'+name_algorithm+'/'+problem+'_'+name_oprt+'_F.npy')
            #print(Mega_X.shape)
            #print(Mega_F.shape)
            if np.any(Mega_X.shape[:3]!= (20, 250, 100)): 
                count_error_shape+=1 
                print(problem, name_algorithm, name_oprt,'Error shape')
            if np.any(Mega_F.shape[:3]!= (20, 250, 100)): 
                print(problem, name_algorithm, name_oprt, 'Error shape')
            if np.any(np.isnan(Mega_X))!=0: 
                count_missing +=1
                print(problem, name_oprt, Mega_X.shape, np.sum(np.isnan(Mega_X)))
            if np.any(np.isnan(Mega_F))!=0: 
                print(problem, name_oprt, Mega_F.shape, np.sum(np.isnan(Mega_F)))
    print('Total of Datasets with missingsproblems: ', count_missing)
    print('Total of Datasets with incorrection shape: ', count_error_shape)
            

In [5]:
algorithms = ['nsgaii'] #'moead' 'age_moea', 'nsgaiii' 'nsgaii'
methods = ['baseline', 'sam_energy', 'sam_uniform','sam_redd', 'sam_mssd',
           'mut_dirichlet', 'mut_exponential', 'mut_normal', 'mut_lognormal']
for algorithm in algorithms: 
    print(algorithm)
    checkingNaNs(wfg_problems+dtlz_problems+zdt_problems, methods,  algorithm)

nsgaii
Total of Datasets with missingsproblems:  0
Total of Datasets with incorrection shape:  0
